In [2]:
import urllib
f = urllib.urlretrieve("http://malifauzi.lecture.ub.ac.id/files/2019/05/DatasetKlasifikasi.zip", "klasifikasi.zip")

### Baca data training dan testing dari file 

In [36]:
dataset_train = spark.read.format("csv").options(header='true', inferschema='true').load("DatasetKlasifikasi/heart_train.csv")
dataset_test = spark.read.format("csv").options(header='true', inferschema='true').load("DatasetKlasifikasi/heart_test.csv")
dataset_train.show()
dataset_test.show()

+---+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
| id|age|sex| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|
+---+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
|  1| 63|  1|  3|     145| 233|  1|      0|    150|    0|    2.3|    0|  0|   1|     1|
|  2| 37|  1|  2|     130| 250|  0|      1|    187|    0|    3.5|    0|  0|   2|     1|
|  3| 41|  0|  1|     130| 204|  0|      0|    172|    0|    1.4|    2|  0|   2|     1|
|  4| 56|  1|  1|     120| 236|  0|      1|    178|    0|    0.8|    2|  0|   2|     1|
|  5| 57|  0|  0|     120| 354|  0|      1|    163|    1|    0.6|    2|  0|   2|     1|
|  6| 57|  1|  0|     140| 192|  0|      1|    148|    0|    0.4|    1|  0|   1|     1|
|  7| 56|  0|  1|     140| 294|  0|      0|    153|    0|    1.3|    1|  0|   2|     1|
|  8| 44|  1|  1|     120| 263|  0|      1|    173|    0|    0.0|    2|  0|   3|     1|
|  9| 52|  1|  2|     172| 199| 

### Menentukan fitur dari Data Tarining

In [37]:
from pyspark.ml.feature import VectorAssembler

vecAssembler = VectorAssembler(inputCols=["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal" ], outputCol="features")
new_dataset_train = vecAssembler.transform(dataset_train)
new_dataset_train.show()

+---+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+--------------------+
| id|age|sex| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|            features|
+---+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+--------------------+
|  1| 63|  1|  3|     145| 233|  1|      0|    150|    0|    2.3|    0|  0|   1|     1|[63.0,1.0,3.0,145...|
|  2| 37|  1|  2|     130| 250|  0|      1|    187|    0|    3.5|    0|  0|   2|     1|[37.0,1.0,2.0,130...|
|  3| 41|  0|  1|     130| 204|  0|      0|    172|    0|    1.4|    2|  0|   2|     1|[41.0,0.0,1.0,130...|
|  4| 56|  1|  1|     120| 236|  0|      1|    178|    0|    0.8|    2|  0|   2|     1|[56.0,1.0,1.0,120...|
|  5| 57|  0|  0|     120| 354|  0|      1|    163|    1|    0.6|    2|  0|   2|     1|[57.0,0.0,0.0,120...|
|  6| 57|  1|  0|     140| 192|  0|      1|    148|    0|    0.4|    1|  0|   1|     1|[57.0,1.0,0.0,140...|
|  7| 56|  0|  1|  

### memilih Algoritma

In [38]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(smoothing=1.0, modelType="multinomial", labelCol='target')

### Membuat model

In [39]:
model = nb.fit(new_dataset_train)

### Menentukan fitur data testing

In [40]:
from pyspark.ml.feature import VectorAssembler

vecAssembler = VectorAssembler(inputCols=["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal" ], outputCol="features")
new_dataset_test = vecAssembler.transform(dataset_test)
new_dataset_test.show()

+---+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+--------------------+
| id|age|sex| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|            features|
+---+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+--------------------+
|201| 44|  1|  0|     110| 197|  0|      0|    177|    0|    0.0|    2|  1|   2|     0|[44.0,1.0,0.0,110...|
|202| 60|  1|  0|     125| 258|  0|      0|    141|    1|    2.8|    1|  1|   3|     0|[60.0,1.0,0.0,125...|
|203| 58|  1|  0|     150| 270|  0|      0|    111|    1|    0.8|    2|  0|   3|     0|[58.0,1.0,0.0,150...|
|204| 68|  1|  2|     180| 274|  1|      0|    150|    1|    1.6|    1|  0|   3|     0|[68.0,1.0,2.0,180...|
|205| 62|  0|  0|     160| 164|  0|      0|    145|    0|    6.2|    0|  3|   3|     0|(13,[0,3,4,7,9,11...|
|206| 52|  1|  0|     128| 255|  0|      1|    161|    1|    0.0|    2|  1|   3|     0|[52.0,1.0,0.0,128...|
|207| 59|  1|  0|  

### Melakukan prediksi pada data testing

In [41]:
# select example rows to display.
predictions_result = model.transform(new_dataset_test)
predictions_result.select('id', 'target', 'prediction').show()

+---+------+----------+
| id|target|prediction|
+---+------+----------+
|201|     0|       1.0|
|202|     0|       0.0|
|203|     0|       0.0|
|204|     0|       0.0|
|205|     0|       0.0|
|206|     0|       1.0|
|207|     0|       0.0|
|208|     0|       0.0|
|209|     0|       1.0|
|210|     0|       1.0|
|211|     0|       1.0|
|212|     0|       0.0|
|213|     0|       1.0|
|214|     0|       0.0|
|215|     0|       0.0|
|216|     0|       0.0|
|217|     0|       0.0|
|218|     0|       0.0|
|219|     0|       0.0|
|220|     0|       0.0|
+---+------+----------+
only showing top 20 rows



### Melakukan evaluasi

In [43]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="target", metricName="areaUnderPR")
evaluator.evaluate(predictions_result)

0.0

In [46]:
saved_prediction = predictions_result.select('id', 'target', 'prediction')
saved_prediction.write.csv("/home/hduser/Documents/HasilKlasifikasi", header = 'true')